<a href="https://colab.research.google.com/github/scharnk/madam_marie/blob/main/BiddyTarot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install retrying

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=a82191969ac8b7aba1fef60e9ffa799b4a9d6bbe4257a2a6e2960337e58148a3
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [2]:
import seaborn as sns
sns.set()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
# import requests_cache
import dill
from bs4 import BeautifulSoup
from datetime import datetime
from retrying import retry
import re
import spacy
import itertools  # itertools.combinations may be useful
import networkx as nx

In [3]:
url_list = ['https://www.biddytarot.com/tarot-card-meanings/major-arcana/', 
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/']

In [4]:
url_major_arcana = [
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/fool/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/magician/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/high-priestess/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/empress/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/emperor/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hierophant/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/lovers/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/chariot/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/strength/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hermit/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/wheel-of-fortune/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/justice/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hanged-man/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/death/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/temperance/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/devil/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/tower/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/star/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/moon/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/sun/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/judgement/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/world/'
]

In [5]:
url_minor_arcana = [
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/ace-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/two-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/three-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/four-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/five-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/six-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/seven-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/eight-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/nine-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/ten-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/page-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/knight-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/queen-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/king-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/ace-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/two-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/three-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/four-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/five-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/six-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/seven-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/eight-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/nine-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/ten-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/page-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/knight-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/queen-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/king-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/ace-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/two-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/three-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/four-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/five-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/six-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/seven-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/eight-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/nine-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/ten-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/page-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/knight-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/queen-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/king-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/ace-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/two-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/three-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/four-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/five-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/six-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/seven-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/eight-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/nine-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/ten-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/page-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/knight-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/queen-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/king-of-wands/'
]

In [6]:
card_list = [
             "Fool",
             "The Magician",
             "High Priestess",
             "The Empress",
             "The Emperor",
             "Hierophant",
             "Lovers",
             "Chariot",
             "Strength",
             "Hermit",
             "Wheel of Fortune",
             "Justice",
             "Hanged Man",
             "Death",
             "Temperance",
             "Devil",
             "Tower",
             "Star",
             "Moon",
             "Sun",
             "Judgement",
             "World"
             ]

In [8]:
for i, v in enumerate(url_major_arcana):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [7]:
#run through url_major_arcana to get full card meanings for upright and reversed

major_meanings = []

for idx, url in enumerate(url_major_arcana):
    card = requests.get(url_major_arcana[idx])
    soup = BeautifulSoup(card.text, "lxml")
    p = soup.find_all(id = ["upright", "reversed"])
    text_p = []
    for i in p:
        text_p.append(i.getText())
    major_meanings.append(text_p)

#run through url_minor_arcana to get full card meanings for upright and reversed

minor_meanings_upright = []
minor_meanings_reversed = []

for idx, url in enumerate(url_minor_arcana):
    card = requests.get(url_minor_arcana[idx])
    soup = BeautifulSoup(card.text, "lxml")
    p = soup.find_all('div', {'class':'col span_12'})
    q = soup.find_all('div', {'class':'col span_8'})
    text_p = []
    text_q = []
    for i in p:
        # p.find('class'='col span_12')
        text_p.append(i.getText())
    minor_meanings_upright.append(text_p[-1])
    for i in q:
        text_q.append(i.getText())
    minor_meanings_reversed.append(text_q[-1])


#run through url list to get keywords for each card upright and reversed


#can figure out how to get general card descriptions later on... should be similar to card meanings




In [10]:
import pickle

In [ ]:
# Save meanings so you don't have to re-scrape biddytarot.com

with open('major_meanings.pkl', 'wb') as f:
     pickle.dump(major_meanings, f)

with open('minor_meanings_upright.pkl', 'wb') as f:
     pickle.dump(minor_meanings_upright, f)

with open('minor_meanings_reversed.pkl', 'wb') as f:
     pickle.dump(minor_meanings_reversed, f)

In [ ]:
# Read the meanings from the dill/pickle files above

with open('major_meanings.pkl', 'rb') as f:
     major_meanings = pickle.load(f)

with open('minor_meanings_upright.pkl', 'rb') as f:
     minor_meanings_upright = pickle.load(f)

with open('minor_meanings_reversed.pkl', 'rb') as f:
     minor_meanings_reversed = pickle.load(f)

In [ ]:
# minor_meanings = []
# text_p = []


In [ ]:
len(minor_meanings_upright)

In [ ]:
minor_meanings_upright

In [ ]:
len(minor_meanings_reversed)

In [ ]:
# major_meanings[3]

In [11]:
page = requests.get('https://www.biddytarot.com/tarot-card-meanings/') # Use requests.get to download the page.

In [12]:
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
# url_base = 'https://www.biddytarot.com/'
# url_end = ['/tarot-card-meanings/major-arcana/', '/tarot-card-meanings/minor-arcana/suit-of-cups/', '/tarot-card-meanings/minor-arcana/suit-of-swords/', '/tarot-card-meanings/minor-arcana/suit-of-pentacles/', '/tarot-card-meanings/minor-arcana/suit-of-wands/']

In [ ]:
# soup.prettify

In [8]:
page = requests.get(url_list[0]) # Use requests.get to download the page.
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [ ]:
# def get_links(response):
#     soup = BeautifulSoup(response.text, "lxml")
#     links = soup.select('div[a]')
#     pairs = []
#     for i in range(len(links)):
#         pair = get_link_date(links[i])
#         pairs.append(pair)
#     return pairs # A list of URL, date pairs
#     return links
    # for li in div:
    # all_li = li.find_all('li')
    # for link in all_li.find_all('a'):
    #     print(link.get('href'))

In [ ]:
# cups = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/')
# soup_cups = BeautifulSoup(cups.text,"lxml")
# a = soup_cups.find_all('a')
# for link in a:
#     print(link.get('href'))

In [ ]:
# swords = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/')
# soup_swords = BeautifulSoup(swords.text,"lxml")
# a = soup_swords.find_all('a')
# for link in a:
#     print(link.get('href'))

In [ ]:
# pentacles = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/')
# soup_pentacles = BeautifulSoup(pentacles.text,"lxml")
# a = soup_pentacles.find_all('a')
# for link in a:
#     print(link.get('href'))

In [ ]:
# wands = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/')
# soup_wands = BeautifulSoup(wands.text,"lxml")
# a = soup_wands.find_all('a')
# for link in a:
#     print(link.get('href'))

In [43]:
# page = requests.get(url_list[0]) # Use requests.get to download the page.

# url_list = ['https://www.biddytarot.com/tarot-card-meanings/major-arcana/', 
#             'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/',
#             'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/',
#             'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/',
#             'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/']

# soup = BeautifulSoup(page.text, "html.parser")

# maj = requests.get('https://www.biddytarot.com/tarot-card-meanings/major-arcana/')

# for i,v in enumerate(url_major_arcana):
#     # print(url_major_arcana[i])
#     maj = requests.get({}.format(url_major_arcana[i]))
#     soup_maj = BeautifulSoup(maj.text,"lxml")
#     divtag = soup_maj.find_all("div", {"class":"div.col.span_12"})


# single instance
# for i,v in enumerate(url_major_arcana):
    # print(url_major_arcana[i])
maj = requests.get("{}".format(url_major_arcana[0]))
soup_maj = BeautifulSoup(maj.text,"lxml")
divtag = soup_maj.find_all("div", {"class":"col span_12"})
divtagrev = soup_maj.find_all("div", {"class":"col span_8"})
divtagdesc = soup_maj.find_all("h3", {"class":"center lightpurple upper fs24 bold padbod15"})

meaning_upright = divtag[2].getText().replace("Upright","Upright: ")
meaning_reversed = divtagrev[1].getText().replace("Reversed","Reversed: ", 1)

meaning_desc = divtagrev[0].getText()
desc = meaning_desc.find("Description")
card_desc = meaning_desc[desc:].replace("Description", "Description: ")
# description = <h3 class="center lightpurple upper fs24 bold padbot15">The Magician Description</h3>
print(meaning_reversed)

######## divtag[2] includes UPRIGHT meaning ############
######## divtagrev[0] includes UPRIGHT keywords, REVERSED keywords, and DESCRIPTION ############
######## don't need UPRIGHT keywords, or REVERSED keywords -- taken from major_arcana page ############
######## divtagrev[1] includes REVERSED meaning ############

######## could build upright meanings up in one array, and reversed in another ########
######## then create dictionary with cards and items as upright, reversed, and description? ########



    # a = soup_maj.find_all('a')
    # for link in a:
    #     print(link.get('href'))

# text_p = []
# p = soup_maj.find_all('p')
# # text_p = [text_p.append(i.getText()) for i in p]
# text_p = []
# for i in p:
#     text_p.append(i.getText())
# print(len(text_p))


###############################################################################
# soup = BeautifulSoup(html)
# divTag = soup.find_all("div", {"class": "tablebox"})

# for tag in divTag:
#     tdTags = tag.find_all("td", {"class": "align-right"})
#     for tag in tdTags:
#         print tag.text

The Fool Reversed:  The Fool Reversed suggests that you have conceived of a new project but aren’t ready to ‘birth’ it into the world just yet. You may worry that you are not fit or that you don’t have all the tools, skills and resources you need to make this project a success. Or perhaps you have a sense that the timing isn’t right. Something is holding you back, and you are preventing yourself from moving forward. You may choose to keep this new opportunity to yourself for now, or you might be ‘parking’ it until a better time.You may fear the unknown, wondering, ‘What am I getting myself into?’ As a result, you have come to a standstill, worried about taking any action where you don’t know the outcome. This often boils down to the need to control everything. Balance this out with knowing that the Universe has your back and you can take this step forward, even if you are unsure of exactly what will happen next.On the flipside, The Fool Reversed can show that you are taking too many ri

In [33]:
# text_p = []
# p = soup_maj.find_all(id = ["upright", "reversed"])
# text_p = [text_p.append(i.getText()) for i in p]
# for i in p:
    # text_p.append(i.getText())

In [34]:
p = soup_maj.find_all(id = ["upright", "reversed"])

In [35]:
text_p

[]

In [13]:
text_p = []
p = soup_maj.find_all('p')
# text_p = [text_p.append(i.getText()) for i in p]
text_p = []
for i in p:
    text_p.append(i.getText())
print(len(text_p))

57


In [46]:
a = soup.find_all('p')

In [ ]:
# a = soup_maj.find(id=["upright","reversed"])

In [50]:
print(p)

[<p class="center bold padbot0 purple">The Easy Way to Interpret The Cards</p>, <p>The Major Arcana Tarot cards represent the life lessons, karmic influences and the big archetypal themes that are influencing your life and your soul's journey to enlightenment. The Major Arcana card meanings are deep and complex –in beautiful ways! Theses Tarot cards truly represent the structure of human consciousness and hold the keys to life lessons passed down through the ages.</p>, <p>Also known as the Trump Cards, the Major Arcana cards include 21 numbered cards and 1 unnumbered card (the Fool). The Fool is the main character of the Major Arcana and makes his journey through each of the cards, meeting new teachers and learning new life lessons along the way, and eventually reaching the completion of his journey with the World card. This is known as the Fool's Journey and is a helpful way of understanding the story line of the Major Arcana Tarot card meanings.</p>, <p>When you see a Major Arcana ca

In [57]:
# from BeautifulSoup import BeautifulSoup

VALID_TAGS = ['div', 'p']

soup_div_p = BeautifulSoup(maj.text,"lxml")
# soup_maj = BeautifulSoup(maj.text,"lxml")
for tag in soup_div_p.find_all('div/div/p'):
    if tag.name not in VALID_TAGS:
        tag.replace_with(tag.renderContents())

print(soup_div_p.renderContents())
# R

b'<!DOCTYPE html>\n<html class="no-js" lang="en-US"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=0" name="viewport"/><link href="https://www.biddytarot.com/wp-content/uploads/BT-BiddyTarotLogo-@2x.png" rel="shortcut icon"/><meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/> <script data-cfasync="false" type="text/javascript">var _vwo_clicks = 10;</script>\n<script data-cfasync="false" type="text/javascript">window._vwo_code = window._vwo_code || (function(){\n    var account_id= 563498,\n    settings_tolerance= 2000,\n    library_tolerance= 2500,\n    use_existing_jquery= false,\n    is_spa=1,\n    hide_element=\'body\',\n    /* DO NOT EDIT BELOW THIS LINE */\n    f=false,d=document,code={use_existing_jquery:function(){return use_existing_jquery;},library_tolerance:function(){return library_tolerance;},finish:function(){if(!f){f=true;var a=d.getElementById(\'_vis_

In [21]:
major_page = requests.get('https://www.biddytarot.com/tarot-card-meanings/major-arcana/')

In [22]:
soup_major = BeautifulSoup(major_page.text, "lxml")

In [23]:
items = soup_major.find_all('div', {'class' : 'card_item'})

In [24]:
keywords = []
for i in items:
  keywords.append(i.getText())

In [25]:
keywords


['    UprightBeginnings, innocence, spontaneity, a free spiritReversedHolding back, recklessness, risk-takingThe Fool   ',
 '    UprightManifestation, resourcefulness, power, inspired actionReversedManipulation, poor planning, untapped talentsThe Magician   ',
 '    UprightIntuition, sacred knowledge, divine feminine, the subconscious mindReversedSecrets, disconnected from intuition, withdrawal and silenceThe High Priestess   ',
 '    UprightFemininity, beauty, nature, nurturing, abundanceReversedCreative block, dependence on othersThe Empress   ',
 '    UprightAuthority, establishment, structure, a father figureReversedDomination, excessive control, lack of discipline, inflexibilityThe Emperor   ',
 '    UprightSpiritual wisdom, religious beliefs, conformity, tradition,institutionsReversedPersonal beliefs, freedom, challenging the status quoThe Hierophant   ',
 '    UprightLove, harmony, relationships, values alignment, choicesReversedSelf-love, disharmony, imbalance, misalignment of 

In [26]:
keywords_test = [i.split('Reversed') for i in keywords] 

In [27]:
keywords_test

[['    UprightBeginnings, innocence, spontaneity, a free spirit',
  'Holding back, recklessness, risk-takingThe Fool   '],
 ['    UprightManifestation, resourcefulness, power, inspired action',
  'Manipulation, poor planning, untapped talentsThe Magician   '],
 ['    UprightIntuition, sacred knowledge, divine feminine, the subconscious mind',
  'Secrets, disconnected from intuition, withdrawal and silenceThe High Priestess   '],
 ['    UprightFemininity, beauty, nature, nurturing, abundance',
  'Creative block, dependence on othersThe Empress   '],
 ['    UprightAuthority, establishment, structure, a father figure',
  'Domination, excessive control, lack of discipline, inflexibilityThe Emperor   '],
 ['    UprightSpiritual wisdom, religious beliefs, conformity, tradition,institutions',
  'Personal beliefs, freedom, challenging the status quoThe Hierophant   '],
 ['    UprightLove, harmony, relationships, values alignment, choices',
  'Self-love, disharmony, imbalance, misalignment of v

In [29]:
# keywords3

In [30]:
print(items[0].getText())

    UprightBeginnings, innocence, spontaneity, a free spiritReversedHolding back, recklessness, risk-takingThe Fool   
